In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

In [15]:
from collections import Counter
import sys
import time
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model

## Load the Dataset

Load the preprocessed data as stored in Numpy-files. Please note that the data has already been split up in a training (training), validation (val), and test subsets.

In [5]:
d_name = 'PAM2'
num_classes, sensors, locations, label_names, f_hz, dimensions, path = get_details(d_name)

print(sensors)
print(locations)
print(f_hz)

['acc', 'gyr', 'mag']
['wrist', 'ankle', 'chest']
100


In [6]:
X_train0, y_train_binary, X_val0, y_val_binary, X_test0, y_test_binary = load_dataset(d_name, path, num_classes)
print(X_train0.shape)
print(X_test0.shape)
print(X_val0.shape)
print(y_train_binary.shape)
print(y_test_binary.shape)
print(y_val_binary.shape)

(14438, 512, 27)
(2180, 512, 27)
(200, 512, 27)
(14438, 12)
(2180, 12)
(200, 12)


In [13]:
y_train = np.argmax(y_train_binary, axis=1)
y_test = np.argmax(y_test_binary, axis=1)
y_tval = np.argmax(y_val_binary, axis=1)

Counter(y_train)

Counter({1: 1411,
         10: 1883,
         2: 1506,
         6: 1471,
         8: 553,
         5: 1251,
         11: 314,
         0: 1516,
         3: 1897,
         7: 686,
         9: 1337,
         4: 613})

## My Models

In [9]:
from existing_models import *

In [36]:
#network_type = 'MLP'
#network_type = 'CNN'
#network_type = 'LSTM'
network_type = 'ConvLSTM'


In [37]:
print('Reshaping data for different models ...')
X_train, X_val, X_test = reshape_data(X_train0, X_val0, X_test0, network_type)

Reshaping data for different models ...
ConvLSTM


In [38]:
#specifying hyper-parameters
batch_size = 256
_, win_len, dim = X_train0.shape

In [39]:
print('building the model ...')
if network_type =='CNN' :
    model = model_CNN(dim, win_len, num_classes, num_feat_map=32, p=0.3)

if network_type =='ConvLSTM':
    model = model_ConvLSTM(dim, win_len, num_classes, num_feat_map=32, p=0.3)

if network_type =='LSTM':
    model = model_LSTM(dim, win_len, num_classes, num_hidden_lstm=32, p=0.3)
    
if network_type =='MLP': 
    model = model_MLP(dim, win_len, num_classes, num_hidden_mlp=256, p=0.3)

print(model.summary())

building the model ...
Model: "ConvLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 27, 512, 32)       128       
_________________________________________________________________
Bn_1 (BatchNormalization)    (None, 27, 512, 32)       128       
_________________________________________________________________
Max_pool_1 (MaxPooling2D)    (None, 27, 256, 32)       0         
_________________________________________________________________
Drop_1 (Dropout)             (None, 27, 256, 32)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 27, 256, 32)       3104      
_________________________________________________________________
Bn_2 (BatchNormalization)    (None, 27, 256, 32)       128       
_________________________________________________________________
Max_pool_2 (MaxPooling2D)    (None,

In [40]:
print('model training ...')
epochs = 10
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model_dir = f'Models/{d_name}'

import os
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

name = '{}_{}'.format(network_type, int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs\{}'.format(name))

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# checkpoint
filepath= f"best_{name}.hdf5"
chk_path = os.path.join(model_dir, filepath)
checkpoint = ModelCheckpoint(chk_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model.fit(X_train, y_train_binary,
          batch_size=300,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=(X_val, y_val_binary),
          callbacks=[tensorboard, checkpoint])

model.save(os.path.join(model_dir,f'final_{name}.hdf5'))

model training ...
Train on 14438 samples, validate on 200 samples
Epoch 1/10
14400/14438 [============================>.] - ETA: 0s - loss: 1.4598 - accuracy: 0.5787
Epoch 00001: val_accuracy improved from -inf to 0.56500, saving model to Models/PAM2\best_ConvLSTM_1604311341.hdf5
14438/14438 [==============================] - 8s 578us/sample - loss: 1.4582 - accuracy: 0.5793 - val_loss: 1.5511 - val_accuracy: 0.5650
Epoch 2/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.6428 - accuracy: 0.8378
Epoch 00002: val_accuracy improved from 0.56500 to 0.79000, saving model to Models/PAM2\best_ConvLSTM_1604311341.hdf5
14438/14438 [==============================] - 6s 434us/sample - loss: 0.6425 - accuracy: 0.8379 - val_loss: 0.7566 - val_accuracy: 0.7900
Epoch 3/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.3700 - accuracy: 0.9170
Epoch 00003: val_accuracy improved from 0.79000 to 0.81000, saving model to Models/PAM2\best_ConvLSTM_1604311341.hdf5
1443

In [41]:
model = load_model(chk_path)

In [42]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test_binary, axis=1)
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
class_wise_f1 = f1_score(y_true, y_pred, average=None)
print('the mean-f1 score: {:.4f}'.format(np.mean(class_wise_f1)))
accuracy = accuracy_score(y_true, y_pred)
print('accuracy is: {:.4f}'.format(accuracy))

[[195   3   0   0   0   0   0   0   0   0   0   0]
 [  0 222   0   0   0   0   0   0   0   0   0   0]
 [  3  20 150   0   0   0   0   0   0   0   7   0]
 [  0   0   0 269   0   0   0   0   0   0   0   0]
 [  0   0   0   0 203   0   0   0   0   0   0   0]
 [  0   0   0   0   0 189   0   7   5   0   0   0]
 [  0   0   0   0   0   0 219   0   0   0   0   0]
 [  0   0   0   0   0   0   0  79   0   0   0   0]
 [  0   0   0   3   0   0   3  18  50   0   0   0]
 [  0   5   1   0   0   0   0   0   0 188   7   0]
 [  0  31  15   0   0   0   0   0   0   0 239   0]
 [  0   0   0   0   6   0   3   0   0   0   0  40]]
the mean-f1 score: 0.9219
accuracy is: 0.9372


## All Cases

In [13]:
network_types = ['MLP', 'CNN', 'LSTM', 'ConvLSTM']
b_list = [False, True]
d_list = [False, True]
results = []

for network_type in network_types:
    for b in b_list:
        for d in d_list:
            print('Reshaping data for different models ...')
            X_train, X_val, X_test = reshape_data(X_train0, X_val0, X_test0, network_type)

            #specifying hyper-parameters
            batch_size = 300
            _, win_len, dim = X_train0.shape

            print('building the model ...')
            if network_type =='CNN' :
                model = model_CNN(dim, win_len, num_classes, num_feat_map=32, p=0.3, batchnorm=b, dropout=d)

            if network_type =='ConvLSTM':
                model = model_ConvLSTM(dim, win_len, num_classes, num_feat_map=32, p=0.3, batchnorm=b, dropout=d)

            if network_type =='LSTM':
                model = model_LSTM(dim, win_len, num_classes, num_hidden_lstm=32, p=0.3, batchnorm=b, dropout=d)

            if network_type =='MLP': 
                model = model_MLP(dim, win_len, num_classes, num_hidden_mlp=256, p=0.3, batchnorm=b, dropout=d)

            print(model.summary())

            print('model training ...')
            epochs = 10
            model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                          optimizer='adam',
                          metrics=['accuracy'])

            model_dir = 'Models/{}'.format(d_name)

            name = '{}_d{}_bn{}'.format(network_type, d, b)
            tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name))

            if not os.path.exists(model_dir):
                os.makedirs(model_dir)

            # checkpoint
            filepath= "best_{}.hdf5".format(name)
            chk_path = os.path.join(model_dir, filepath)
            checkpoint = ModelCheckpoint(chk_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

            model.fit(X_train, y_train_binary,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      shuffle=True,
                      validation_data=(X_val, y_val_binary),
                      callbacks=[tensorboard, checkpoint])

            model.save(os.path.join(model_dir,'final_{}.hdf5'.format(name)))
            
            model = load_model(chk_path)

            y_pred = np.argmax(model.predict(X_test), axis=1)
            y_true = np.argmax(y_test_binary, axis=1)
            cf_matrix = confusion_matrix(y_true, y_pred)
            class_wise_f1 = f1_score(y_true, y_pred, average=None)
            accuracy = accuracy_score(y_true, y_pred)
            results.append((network_type, b, d, cf_matrix, np.mean(class_wise_f1), accuracy))

Reshaping data for different models ...
MLP
building the model ...
Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               3539200   
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_out (Dense)            (None, 12)                3084      
Total params: 3,608,076
Trainable params: 3,608,076
Non-trainable params: 0
_________________________________________________________________
None
model training ...
Train on 14438 samples, validate on 200 samples
Epoch 1/10
13800/14438 [===========================>..] - ETA: 0s - loss: 29.8426 - acc: 0.5314
Epoch 00001: val_acc improved from -inf to 0.67000, saving model to Models/PAM2/best_MLP_dFalse_bnFalse.hdf5
14438/14438 [==========================

/home/vikranth/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Reshaping data for different models ...
MLP
building the model ...
Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               3539200   
_________________________________________________________________
Bn_1 (BatchNormalization)    (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
Bn_2 (BatchNormalization)    (None, 256)               1024      
_________________________________________________________________
dense_out (Dense)            (None, 12)                3084      
Total params: 3,610,124
Trainable params: 3,609,100
Non-trainable params: 1,024
_________________________________________________________________
None
model training ...
Train on 14438 samples, 

/home/vikranth/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Reshaping data for different models ...
MLP
building the model ...
Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               3539200   
_________________________________________________________________
Bn_1 (BatchNormalization)    (None, 256)               1024      
_________________________________________________________________
Drop_1 (Dropout)             (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
Bn_2 (BatchNormalization)    (None, 256)               1024      
_________________________________________________________________
Drop_2 (Dropout)             (None, 256)               0         
______________________________________________________________

...

/home/vikranth/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Reshaping data for different models ...
LSTM
building the model ...
Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Lstm_1 (LSTM)                (None, 512, 32)           7680      
_________________________________________________________________
Bn_1 (BatchNormalization)    (None, 512, 32)           128       
_________________________________________________________________
Lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
Bn_2 (BatchNormalization)    (None, 32)                128       
_________________________________________________________________
dense_out (Dense)            (None, 12)                396       
Total params: 16,652
Trainable params: 16,524
Non-trainable params: 128
_________________________________________________________________
None
model training ...
Train on 14438 samples, valida

Epoch 10/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.6669 - acc: 0.7907
Epoch 00010: val_acc did not improve from 0.67000
14438/14438 [==============================] - 57s 4ms/sample - loss: 0.6667 - acc: 0.7908 - val_loss: 1.3138 - val_acc: 0.6500
Reshaping data for different models ...
ConvLSTM
building the model ...
Model: "ConvLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 27, 512, 32)       128       
_________________________________________________________________
Max_pool_1 (MaxPooling2D)    (None, 27, 256, 32)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 27, 256, 32)       3104      
_________________________________________________________________
Max_pool_2 (MaxPooling2D)    (None, 27, 128, 32)       0         
__________________________________________

Epoch 5/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.6704 - acc: 0.8304
Epoch 00005: val_acc improved from 0.67000 to 0.76000, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnFalse.hdf5
14438/14438 [==============================] - 13s 902us/sample - loss: 0.6703 - acc: 0.8304 - val_loss: 0.7557 - val_acc: 0.7600
Epoch 6/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.5612 - acc: 0.8585
Epoch 00006: val_acc improved from 0.76000 to 0.80500, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnFalse.hdf5
14438/14438 [==============================] - 13s 904us/sample - loss: 0.5613 - acc: 0.8585 - val_loss: 0.6969 - val_acc: 0.8050
Epoch 7/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.4582 - acc: 0.8913
Epoch 00007: val_acc improved from 0.80500 to 0.81000, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnFalse.hdf5
14438/14438 [==============================] - 13s 899us/sample - loss: 0.4582 - acc: 0.8912 - val_loss: 0.

Epoch 1/10
14400/14438 [============================>.] - ETA: 0s - loss: 1.3076 - acc: 0.6027
Epoch 00001: val_acc improved from -inf to 0.47000, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnTrue.hdf5
14438/14438 [==============================] - 22s 2ms/sample - loss: 1.3064 - acc: 0.6031 - val_loss: 1.5913 - val_acc: 0.4700
Epoch 2/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.5472 - acc: 0.8558
Epoch 00002: val_acc improved from 0.47000 to 0.72000, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnTrue.hdf5
14438/14438 [==============================] - 16s 1ms/sample - loss: 0.5467 - acc: 0.8559 - val_loss: 0.9200 - val_acc: 0.7200
Epoch 3/10
14400/14438 [============================>.] - ETA: 0s - loss: 0.3251 - acc: 0.9235
Epoch 00003: val_acc improved from 0.72000 to 0.79500, saving model to Models/PAM2/best_ConvLSTM_dTrue_bnTrue.hdf5
14438/14438 [==============================] - 16s 1ms/sample - loss: 0.3249 - acc: 0.9235 - val_loss: 0.5715 - val_a

In [14]:
results

[('MLP',
  False,
  False,
  array([[198,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  2, 189,  26,   1,   0,   0,   4,   0,   0,   0,   0,   0],
         [  0,  23, 140,   5,   0,   0,   3,   0,   3,   0,   6,   0],
         [  0,   0,   2, 249,   0,   0,  10,   0,   8,   0,   0,   0],
         [  0,   7,   0,   0,  92,   0,  88,   6,   0,   0,   0,  10],
         [  0,   2,   0,   0,   0, 173,   7,   0,   4,   0,   0,  15],
         [  0,  16,   0,   2,   0,   0, 200,   0,   1,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   2,  75,   1,   1,   0,   0],
         [  0,   0,   0,   1,   0,   0,   2,   1,  68,   0,   2,   0],
         [  0,   6,   0,   1,   1,  43,  14,   0,  16, 114,   5,   1],
         [  0,  56,  16,   0,   0,   0,  10,   0,   1,  20, 182,   0],
         [  0,   0,   0,   0,  38,   0,   0,  11,   0,   0,   0,   0]]),
  0.7175698559709597,
  0.7706422018348624),
 ('MLP',
  False,
  True,
  array([[198,   0,   0,   0,   0,   0,   0,   0

## TfLite

In [52]:
# Convert the model.

saved_model = load_model(os.path.join(model_dir,'best_CNN_1604310806.hdf5'))
converter = tf.lite.TFLiteConverter.from_keras_model(saved_model)
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(model_dir,"converted_model.tflite"), 'wb') as f:
    f.write(tflite_model)

In [53]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path = os.path.join(model_dir,"converted_model.tflite"))
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
#input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = np.array(np.expand_dims(X_test[0], axis=0), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[4.8183036e-04 3.3311927e-04 2.8963152e-05 1.5839161e-03 4.2729325e-06
  1.9714739e-03 5.9188990e-04 1.0917628e-04 6.3189218e-05 8.3506966e-01
  1.5965338e-01 1.0904269e-04]]


In [54]:
# Test model on test data.
y_pred = []
for i,input_sample in enumerate(X_test):
    input_data = np.array(np.expand_dims(input_sample, axis=0), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    y_pred.append(np.argmax(output_data, axis=1))

In [55]:
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy after quantization : {accuracy}')

Accuracy after quantization : 0.8357798165137614
